# **Add Examples to merge.json**
Add `englis-spanish` examples to `merge.json`

### **Packages**

In [1]:
import time
import translatepy
import pandas as pd
import oxford_api
from oxford_api import Word
from pandarallel import pandarallel

### **Functions**

In [2]:
#return the same word if it can't translate
def en_es(word):
    translator = translatepy.Translator()
    gtrans=translator.translate(word,'es','en')
    palabra=gtrans.result
    translator.clean_cache()
    return palabra

def en_es_list(eg):
    if eg is None:
        return None
    else:
        return [[x,en_es(x)] for x in eg]
    
#translator.example
'''
1. return a list of objects <class 'translatepy.translators.bing.Example'>
    - Note: it doent return list of strings. You need to convert to str() to use them.
2. return empty list [] if it doesnt have examples, but sometimes return None
'''
def expl_transpy(word):
    translator = translatepy.Translator()
    ex=translator.example(word,'es','en')
    if ex is None:
        ex = []
    ex = [str(x) for x in ex[:4]]
    translator.clean_cache()
    return ex

def get_examples(word):
    try:
        Word.get(word)
    except:
        expl = expl_transpy(word)[:4]
        if len(expl)==0:
            expl = None
        return expl
    else:
        expl=Word.examples()[:4]
        if len(expl)<4:
            expl = expl + expl_transpy(word)[:4]
            expl = expl[:4]
        if len(expl)==0:
            expl = None
        return expl

### **Read file merge.json**

In [3]:
df=pd.read_json('../merge.json')
#df=df.sample(n=10)

### **Multiprocessing add english examples**

In [4]:
pandarallel.initialize(nb_workers=100)

ti=time.time()
df['examples']=df['word'].parallel_apply(get_examples)
tf=time.time()
print(f'ok! in {tf-ti} seconds')

INFO: Pandarallel will run on 100 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
ok! in 9.709346294403076 seconds


##### **Save new data with examples(en)**

In [5]:
df.to_json('merge_eg_en.json', orient='records', indent=4)

### **Multiprocessing translate exaples data**

In [6]:
df=pd.read_json('merge_eg_en.json')
ti=time.time()
df['examples']=df['examples'].parallel_apply(en_es_list)
tf=time.time()
print(f'ok! in {tf-ti} seconds')

ok! in 2.9663870334625244 seconds


##### **Save new data with examples(en-es)**

In [7]:
df.to_json('merge_eg.json', orient='records', indent=4)